## This notebook will help you if you want to test your database is working right

### Importing libraries

In [1]:
import pandas as pd
import psycopg2
import sqlalchemy as sa
import yfinance as yf

### Importing database credentials 

Before, you must be sure the config.ini file has the corrrect credentials

In [2]:
import configparser

def read_config():
    config = configparser.ConfigParser()
    config.read('../config.ini')
    return config

conn_data = read_config()
conn_data = conn_data["PostgreSQL"]

host = conn_data["HOST"]
port = conn_data["PORT"]
database = conn_data["DATABASE"]
user = conn_data["USER"]
password = conn_data["PASSWORD"]
schema = conn_data["SCHEMA"]

### Test N°1: testing database with a psycopg2 library. 
### If you get the output: [('companies',)], it's working correctly

In [19]:
table_name = 'companies'

with psycopg2.connect(host=host, port=port, user=user, password=password, database=database) as conn:
    with conn.cursor() as cursor:
        cursor.execute(f"DROP TABLE IF EXISTS {schema}.{table_name};")
        query = f"""
                CREATE TABLE {schema}.{table_name}(
                id_company      SERIAL NOT NULL PRIMARY KEY,
                symbol          VARCHAR(10),
                name            VARCHAR(100)
                );
                """
        cursor.execute(query)

with psycopg2.connect(host=host, port=port, user=user, password=password, database=database) as conn:
    with conn.cursor() as cursor:
        query = f"SELECT table_name FROM information_schema.tables WHERE table_schema = '{schema}';"
        cursor.execute(query)
        print(cursor.fetchall())

[('companies',)]


### Test N°2: testing database with SQLAlchemy and psycopg2

### If you get at the output the data of AAPL between '2023-01-01' and '2023-02-02', it's working correctly

In [20]:
df = yf.download(tickers='AAPL', start='2023-01-01', end='2023-02-02', interval='1d').reset_index()
table_name = 'stock_prices'

#------------DATABASE CONNECTION---------------------------------
from sqlalchemy.engine.url import URL
# build the sqlalchemy URL
url = URL.create(
drivername='postgresql',
host=host,
port=port,
database=database,
username=user,
password=password,
)

engine = sa.create_engine(url)

#------------LOAD STOCK PRICES DATA---------------------------------
df.to_sql(table_name, con=engine, schema=schema, if_exists='append', index=False, method='multi')


with psycopg2.connect(host=host, port=port, user=user, password=password, database=database) as conn:
    with conn.cursor() as cursor:
        query = f"SELECT * FROM {schema}.stock_prices;"
        cursor.execute(query)
        data = cursor.fetchall()

pd.DataFrame(data, columns=df.columns)

[*********************100%%**********************]  1 of 1 completed


21